# Gráfico Função de Michaelis

In [ ]:
import numpy as np
import plotly.graph_objects as go

def michaelis (conc_subst, km, vmax):
    v_reaction = (vmax*conc_subst)/(km+conc_subst)
    return v_reaction

km = 0.38
vmax = 32

s = np.linspace(0, 20, 500)
v = michaelis(s, km, vmax)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=s,
    y=v,
    mode='lines',
))

fig.update_layout(
    title="Michaelis",
    xaxis_title="Concentração do Substrato (mol/L)",
    yaxis_title="Velocidade da reação (mol/L.s)"
)

fig.show()

# Gráfico Função de Michaelis (Km e Vmáx ajustável)

In [3]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

def michaelis (conc_subst, km, vmax):
    v_reaction = (vmax*conc_subst)/(km+conc_subst)
    return v_reaction

s = np.linspace(0, 20, 500)
v = michaelis(s, 0.4, 32)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=s,
    y=v,
    mode='lines',
))

fig.update_layout(
    title="Michaelis",
    xaxis=dict(range=[0, 20]),
    yaxis=dict(range=[0, 50]),
    xaxis_title="Concentração do Substrato (mol/L)",
    yaxis_title="Velocidade da reação (mol/L.s)"
)

app = Dash(__name__)

app.layout = html.Div([
    html.Label("Km:", style={'color': 'white'}),
    dcc.Slider(
        id='km-slider',
        min=0.1,
        max=2.0,
        step=0.1,
        value=0.4
    ),

    html.Br(),

    html.Label("Vmax:", style={'color': 'white'}),
    dcc.Slider(
        id='vmax-slider',
        min=10,
        max=50,
        step=1,
        value=32
    ),
    dcc.Graph(id='Velocidade(concentração)', figure=fig)
])

@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('km-slider', 'value'),
    Input('vmax-slider', 'value')
)

def mudar_imagem(km, vmax):
    s = np.linspace(0, 20, 500)
    v = michaelis(s, km, vmax)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=s,
        y=v,
        mode='lines',
    ))

    fig.update_layout(
        title="Michaelis",
        xaxis=dict(range=[0, 20]),
        yaxis=dict(range=[0, 50]),
        xaxis_title="Concentração do Substrato (mol/L)",
        yaxis_title="Velocidade da reação (mol/L.s)"
    )

    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8020)   

# Grafico em função de temperatura e pH

In [12]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import math


def v0(S, T, pH, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH):
    term1 = 1 / (1 + 10**(pKa - pH) + 10**(pH - pKb))
    exp1 = math.exp((-Ea*1000 / 8.314) * (1/(T+273) - 1/(T_ref+273)))
    exp2 = math.exp((deltaH*1000 / 8.314) * (1/(T+273) - 1/(T_ref+273)))
    
    numerador = Vmax_ref * exp1 * S
    denominador = Km_ref * exp2 + S
    
    return term1 * (numerador / denominador)

s = np.linspace(0, 20, 500)
v = v0(s, 25, 7.5, 6.5, 8, 32, 50, 25, 0.4, -20)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=s,
    y=v,
    mode='lines',
))

fig.update_layout(
    title="Michaelis",
    xaxis=dict(range=[0, 20]),
    yaxis=dict(range=[0, 50]),
    xaxis_title="Concentração do Substrato (mol/L)",
    yaxis_title="Velocidade da reação (mol/L.s)"
)

app = Dash(__name__)

app.layout = html.Div([
    html.Label("Constante de dissociação do grupo ácido (pKa)", style={'color': 'white'}),
    dcc.Input(
        id='input_pka',
        type='number',
        placeholder="Insira o valor de pKa",
        value='6.5'),

    html.Br(),

    html.Label("Constante de dissociação do grupo básico (pKb)", style={'color': 'white'}),
    dcc.Input(
        id='input_pkb',
        type='number',
        placeholder="Insira o valor de pKb",
        value='8'),

    html.Br(),

    html.Label("Temperatura referêncial (T_ref)", style={'color': 'white'}),
    dcc.Input(
        id='input_t_ref',
        type='number',
        placeholder="Insira o valor de T_ref",
        value='25'),

    html.Br(),

    html.Label("Velocidade máxima na temperatura referencial (Vmax_ref)", style={'color': 'white'}),
    dcc.Input(
        id='input_vmax_ref',
        type='number',
        placeholder="Insira o valor de Vmax_ref",
        value='32'),

    html.Br(),

    html.Label("Constante de Michaelis na temperatura referencial (Km_ref)", style={'color': 'white'}),
    dcc.Input(
        id='input_km_ref',
        type='number',
        placeholder="Insira o valor de Km_ref",
        value='0.4'),

    html.Br(),

    html.Label("Energia de ativação (Ea)", style={'color': 'white'}),
    dcc.Input(
        id='input_ea',
        type='number',
        placeholder="Insira o valor de Ea",
        value='50'),

    html.Br(),

    html.Label("Variação de Entalpia (ΔH)", style={'color': 'white'}),
    dcc.Input(
        id='input_deltah',
        type='number',
        placeholder="Insira o valor de ΔH",
        value='-20'),

    html.Br(),

    html.Label("Temperatura:", style={'color': 'white'}),
    dcc.Slider(
        id='t-slider',
        min=20,
        max=50,
        step=1,
        value=25
    ),

    html.Br(),

    html.Label("pH:", style={'color': 'white'}),
    dcc.Slider(
        id='ph-slider',
        min=5.5,
        max=9.5,
        step=0.1,
        value=7
    ),
    dcc.Graph(id='Velocidade(concentração)', figure=fig)
])

@app.callback(
    Output('Velocidade(concentração)', 'figure'),
    Input('t-slider', 'value'),
    Input('ph-slider', 'value'),
    Input('input_pka', 'value'),
    Input('input_pkb', 'value'),
    Input('input_t_ref', 'value'),
    Input('input_vmax_ref', 'value'),
    Input('input_km_ref', 'value'),
    Input('input_ea', 'value'),
    Input('input_deltah', 'value'),

)

def mudar_imagem(t, ph, pka_inp, pkb_inp, t_inp, vmax_inp, km_inp, ea_inp, deltah_inp):

    pKa = float(pka_inp)
    pKb = float(pkb_inp)
    Vmax_ref = float(vmax_inp)
    Ea = float(ea_inp)
    T_ref = float(t_inp)
    Km_ref = float(km_inp)
    deltaH = float(deltah_inp)

    s = np.linspace(0, 20, 500)
    v = v0(s, t, ph, pKa, pKb, Vmax_ref, Ea, T_ref, Km_ref, deltaH)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=s,
        y=v,
        mode='lines',
    ))

    fig.update_layout(
        title="Michaelis",
        xaxis=dict(range=[0, 20]),
        yaxis=dict(range=[0, 50]),
        xaxis_title="Concentração do Substrato (mol/L)",
        yaxis_title="Velocidade da reação (mol/L.s)"
    )

    return fig

if __name__ == '__main__':
    app.run(debug=True, port=8043)   

# Em paralelo: concentração em função do tempo

In [13]:
import numpy as np
import plotly.express as px
from math import log

km = 10**(-1)
v_max = 32
s0 = 0.5

# Gerar valores de [S] de s0 até 0.001 - mais próximo de 0
s_final = np.linspace(s0, 0.001, 500)

delta_t = (km*(s0 - s_final) + km *np.log(s0 /s_final))/ v_max

fig = px.line(x=delta_t, y=s_final,
              labels={'x': 'Tempo (s)', 'y': '[S] (M)'},
              title="Variação de [S] de acordo com o tempo")

fig.update_layout(xaxis = dict(range=[0,0.01]), yaxis = dict(range = [0, 1]))

fig.show()
